In [48]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from datetime import datetime
import re

In [49]:
conf=SparkConf().setAppName("Number5").setMaster("local[*]")
sc=SparkContext(conf=conf)

In [50]:
lines_rdd = sc.textFile("/home/jovyan/3_5_Independent_work/posts.csv")

In [51]:
header=lines_rdd.first()
clear_rdd=lines_rdd.filter(lambda line: line != header)

In [52]:
tuple_line=(clear_rdd
                     .map(lambda x: x.split(","))
                     .map(lambda x: (int(x[0]), int(x[2]), int(x[3]), x[1]))
           )
            

In [53]:
print(tuple_line.take(2))

[(101, 268, 16382, 'Carousel'), (102, 138, 9267, 'Reel')]


In [93]:
count_like=tuple_line.map(lambda x: x[2]).reduce(lambda x, y: x + y)
count_comment=tuple_line.map(lambda x: x[1]).reduce(lambda x, y: x + y)

In [127]:
count_post=(tuple_line.map(lambda x: (x[3], 1))
                      .reduceByKey(lambda x, y: x+y)
                      .sortBy(lambda x: x[0])
)
avg_count_like_post=(tuple_line.map(lambda x: (x[3], x[2]))
                               .reduceByKey(lambda x, y: x + y)
                               .join(count_post)
                               .map(lambda x: (x[0], round(x[1][0] / x[1][1], 1)))
                               .sortBy(lambda x: x[1], ascending=False)
)                                                                                   
avg_count_comment_post=(tuple_line.map(lambda x: (x[3], x[1]))
                                  .reduceByKey(lambda x, y: x + y)
                                  .join(count_post)
                                  .map(lambda x: (x[0], round(x[1][0] / x[1][1], 1)))
                                  .sortBy(lambda x: x[1], ascending=False)
)

In [146]:
top_5_like=(tuple_line.map(lambda x: (x[0], x[3], x[2])).top(5, key=lambda x: x[2])
)
top_5_comment=(tuple_line.map(lambda x: (x[0], x[3], x[1])).top(5, key=lambda x: x[2])
)

In [147]:
print(f"Общее количество постов: {tuple_line.count()}")
print(f"Общее количество лайков: {count_like}")
print(f"Общее количество комментариев: {count_comment}")
print(f"Среднее количество лайков на пост: {round(count_like / tuple_line.count(), 1)}")
print(f"Среднее количество комментариев на пост:{round(count_comment / tuple_line.count(), 1)}")

Общее количество постов: 39
Общее количество лайков: 707567
Общее количество комментариев: 8387
Среднее количество лайков на пост: 18142.7
Среднее количество комментариев на пост:215.1


In [148]:
print("Количество постов каждого типа:")
for key, value in count_post.collect():
    print(f"{key}: {value}")
print("Среднее количество лайков по типам постов:")
for key, value in avg_count_like_post.collect():
    print(f"{key}: {value}")
print("Среднее количество комментариев по типам постов:")
for key, value in avg_count_comment_post.collect():
    print(f"{key}: {value}")

Количество постов каждого типа:
Carousel: 14
Image: 11
Reel: 14
Среднее количество лайков по типам постов:
Carousel: 20814.3
Image: 18543.2
Reel: 15156.6
Среднее количество комментариев по типам постов:
Reel: 316.7
Carousel: 174.3
Image: 137.5


In [149]:
print("Топ 5 постов с наибольшим количеством лайков:")
for key, value in avg_count_like_post.collect():
    print(f"{key}: {value}")
print("Топ 5 постов с наибольшим количеством комментариев:")
for key, value in avg_count_comment_post.collect():
    print(f"{key}: {value}")

Топ 5 постов с наибольшим количеством лайков:
Carousel: 20814.3
Image: 18543.2
Reel: 15156.6
Топ 5 постов с наибольшим количеством комментариев:
Reel: 316.7
Carousel: 174.3
Image: 137.5


In [47]:
sc.stop()